In [ ]:
# cspell: ignore ndimage usecols kernelspec ipykernel codemirror ipython nbconvert pygments nbformat #
import numpy as np
from scipy.spatial.transform import Rotation
import scipy.interpolate
from scipy.ndimage import gaussian_filter
import os
import matplotlib.pyplot as plt
import csv
def yaw_transform(raw_yaw):
    transformed_yaw = np.zeros(raw_yaw.shape)
    transformed_yaw[0] = raw_yaw[0]
    for i in range(raw_yaw.shape[0]-1):
        rotate_num = (raw_yaw[i+1]-transformed_yaw[i])//(2*np.pi)
        if raw_yaw[i+1]-transformed_yaw[i] - 2*rotate_num*np.pi < np.pi:
            transformed_yaw[i+1] = raw_yaw[i+1] - 2*rotate_num*np.pi
        else:
            transformed_yaw[i+1] = raw_yaw[i+1] - 2*(rotate_num+1)*np.pi
    return transformed_yaw
def steer_visualize(skip_time = 0,total_time = None,dir_name = ".",ylim = [-0.025,0.025] ):
    kinematic=np.loadtxt(dir_name+"/kinematic_state.csv",delimiter=',',usecols=[0,1,4,5,7,8,9,10,47])
    start_time= kinematic[0,0]+1e-9*kinematic[0,1]
    total_time_default=kinematic[-1,0]+1e-9*kinematic[-1,1]-start_time
    vehicle_steer = np.loadtxt(dir_name+"/steering_status.csv",delimiter=',',usecols=[0,1,2])
    steer = vehicle_steer[:,2]
    control_cmd = np.loadtxt(dir_name+"/control_cmd_orig.csv",delimiter=',',usecols=[0,1,8,16])
    steer_des = control_cmd[:,2]
    plt.plot(control_cmd[:,0] + 1e-9*control_cmd[:,1],steer_des,label="proxima_input")
    plt.plot(vehicle_steer[:,0] + 1e-9*vehicle_steer[:,1],steer,label="observed")
    """
    operation_mode = np.loadtxt(dir_name + "/system_operation_mode_state.csv", delimiter=",", usecols=[0, 1, 2])
    operation_mode_time = np.zeros(2*operation_mode.shape[0])
    operation_mode_time[::2] = operation_mode[:,0] + 1e-9*operation_mode[:,1]
    operation_mode_time[1::2][:-1] = operation_mode_time[2::2]-1e-16
    operation_mode_time[-1] = start_time+total_time_default
    with open(dir_name+"/system_operation_mode_state.csv") as f:
        reader=csv.reader(f,delimiter=',')
        autoware_control_enabled_str=np.array([row[3] for row in reader])
    with open(dir_name+"/system_operation_mode_state.csv") as f:
        reader=csv.reader(f,delimiter=',')
        autonomous_mode_available_str=np.array([row[6] for row in reader])
    autoware_control_enabled = np.zeros(2*operation_mode.shape[0])
    autonomous_mode_available = np.zeros(2*operation_mode.shape[0])
    operation_mode_ = np.ones(2*operation_mode.shape[0])
    for i in range(operation_mode.shape[0]):
        if autoware_control_enabled_str[i] == "True":
            autoware_control_enabled[2*i] = 1
            autoware_control_enabled[2*i+1] = 1

        if operation_mode[i,2] > 1.5:
            operation_mode_[2*i] = 2.0
            operation_mode_[2*i+1] = 2.0
    """
    # plt.plot(operation_mode_time,0.01*operation_mode_,label="operation_mode")
    # plt.plot(operation_mode_time,0.01*autoware_control_enabled,label="autoware_control_enabled")


    if total_time is None:
        plt.xlim([start_time+skip_time,start_time+total_time_default])
    else:
        plt.xlim([start_time+skip_time,start_time+skip_time+total_time])
    plt.ylabel("steer [rad]")
    plt.xlabel("Time [s]")

    plt.ylim(ylim)
    plt.legend()
    plt.show()
def acc_visualize(skip_time = 0,total_time = None,dir_name = ".",ylim = [-0.005,0.02] ):
    kinematic=np.loadtxt(dir_name+"/kinematic_state.csv",delimiter=',',usecols=[0,1,4,5,7,8,9,10,47])
    start_time= kinematic[0,0]+1e-9*kinematic[0,1]
    total_time_default=kinematic[-1,0]+1e-9*kinematic[-1,1]-start_time
    loc_acc = np.loadtxt(dir_name + "/acceleration.csv", delimiter=",", usecols=[0, 1, 3])
    acc = loc_acc[:,2]
    control_cmd = np.loadtxt(dir_name+"/control_cmd_orig.csv",delimiter=',',usecols=[0,1,8,16])
    acc_des = control_cmd[:,3]
    plt.plot(control_cmd[:,0] + 1e-9*control_cmd[:,1],acc_des,label="proxima_input")
    plt.plot(loc_acc[:,0] + 1e-9*loc_acc[:,1],acc,label="observed")
    """
    goal_stop_mode = np.loadtxt(dir_name + "/debug_mpc_goal_stop_mode.csv", delimiter=",",usecols=[0,1])

    with open(dir_name+"/debug_mpc_goal_stop_mode.csv") as f:
        reader=csv.reader(f,delimiter=',')
        stop_mode_str=np.array([row[2] for row in reader])

    stop_mode = np.zeros(goal_stop_mode.shape[0])
    for i in range(goal_stop_mode.shape[0]):
        if stop_mode_str[i] == "True":
            stop_mode[i] = 1
    plt.plot(goal_stop_mode[:,0] + 1e-9*goal_stop_mode[:,1],stop_mode,label="stop_mode")
    operation_mode = np.loadtxt(dir_name + "/system_operation_mode_state.csv", delimiter=",", usecols=[0, 1, 2])
    operation_mode_time = np.zeros(2*operation_mode.shape[0])
    operation_mode_time[::2] = operation_mode[:,0] + 1e-9*operation_mode[:,1]
    operation_mode_time[1::2][:-1] = operation_mode_time[2::2]-1e-16
    operation_mode_time[-1] = start_time+total_time_default
    with open(dir_name+"/system_operation_mode_state.csv") as f:
        reader=csv.reader(f,delimiter=',')
        autoware_control_enabled_str=np.array([row[3] for row in reader])
    with open(dir_name+"/system_operation_mode_state.csv") as f:
        reader=csv.reader(f,delimiter=',')
        autonomous_mode_available_str=np.array([row[6] for row in reader])
    autoware_control_enabled = np.zeros(2*operation_mode.shape[0])
    operation_mode_ = np.ones(2*operation_mode.shape[0])
    for i in range(operation_mode.shape[0]):
        if autoware_control_enabled_str[i] == "True":
            autoware_control_enabled[2*i] = 1
            autoware_control_enabled[2*i+1] = 1

        if operation_mode[i,2] > 1.5:
            operation_mode_[2*i] = 2.0
            operation_mode_[2*i+1] = 2.0
    """
    # plt.plot(operation_mode_time,operation_mode_,label="operation_mode")
    # plt.plot(operation_mode_time,autoware_control_enabled,label="autoware_control_enabled")

    if total_time is None:
        plt.xlim([start_time+skip_time,start_time+total_time_default])
    else:
        plt.xlim([start_time+skip_time,start_time+skip_time+total_time])
    plt.ylabel("acc [m/s^2]")
    plt.xlabel("Time [s]")
    plt.ylim(ylim)
    plt.legend()
    plt.show()
def yaw_visualize(skip_time = 0,total_time = None,dir_name = ".",ylim = [-0.15,0.0]):
    kinematic=np.loadtxt(dir_name+"/kinematic_state.csv",delimiter=',',usecols=[0,1,4,5,7,8,9,10,47])
    raw_yaw = Rotation.from_quat(kinematic[:,4:8]).as_euler("xyz")[:,2]
    yaw = yaw_transform(raw_yaw)
    start_time= kinematic[0,0]+1e-9*kinematic[0,1]
    total_time_default=kinematic[-1,0]+1e-9*kinematic[-1,1]-start_time
    plt.plot(kinematic[:,0] + 1e-9*kinematic[:,1],yaw,label="observed")
    if total_time is None:
        plt.xlim([start_time+skip_time,start_time+total_time_default])
    else:
        plt.xlim([start_time+skip_time,start_time+skip_time+total_time])
    plt.ylabel("yaw [rad]")
    plt.xlabel("Time [s]")

    plt.ylim(ylim)
    plt.legend()
    plt.show()
def vel_visualize(skip_time = 0,total_time = None,dir_name = ".",ylim = [-0.1,12.0]):
    kinematic=np.loadtxt(dir_name+"/kinematic_state.csv",delimiter=',',usecols=[0,1,4,5,7,8,9,10,47])
    vel = kinematic[:,8]
    mpc_v_des=np.loadtxt(dir_name+"/debug_mpc_v_des.csv",delimiter=',',usecols=[0,1,3])
    vel_des = mpc_v_des[:,2]
    start_time= kinematic[0,0]+1e-9*kinematic[0,1]
    total_time_default=kinematic[-1,0]+1e-9*kinematic[-1,1]-start_time
    plt.plot(kinematic[:,0] + 1e-9*kinematic[:,1],vel,label="observed")
    plt.plot(mpc_v_des[:,0] + 1e-9*mpc_v_des[:,1],vel_des,label="des")

    if total_time is None:
        plt.xlim([start_time+skip_time,start_time+total_time_default])
    else:
        plt.xlim([start_time+skip_time,start_time+skip_time+total_time])
    plt.ylabel("vel [m/s]")
    plt.xlabel("Time [s]")

    plt.ylim(ylim)
    plt.legend()
    plt.show()
def lateral_error_visualize(skip_time = 0,total_time = None,dir_name = ".",ylim = [-0.25,0.25],title=None):
    lateral_error = np.loadtxt(dir_name+"/lateral_error.csv",delimiter=',')
    start_time= lateral_error[0,0]+1e-9*lateral_error[0,1]
    total_time_default=lateral_error[-1,0]+1e-9*lateral_error[-1,1]-start_time
    plt.plot(lateral_error[:,0] + 1e-9*lateral_error[:,1],lateral_error[:,2],label="lateral_error")
    if total_time is None:
        plt.xlim([start_time+skip_time,start_time+total_time_default])
    else:
        plt.xlim([start_time+skip_time,start_time+skip_time+total_time])
    plt.legend()
    plt.ylim(ylim)
    plt.ylabel("lateral_error [m]")
    plt.xlabel("Time [s]")
    if title is not None:
        plt.title(title)
        plt.savefig("lateral_error_"+title+".png")
    plt.show()
def computation_time_visualize(skip_time = 0,total_time = None,dir_name = ".",ylim = [-0.05,0.5]):
    control_cmd = np.loadtxt(dir_name+"/control_cmd_orig.csv",delimiter=',',usecols=[0,1])
    cmd_time_stamp = control_cmd[:,0]+1e-9*control_cmd[:,1]
    cmd_time_stamp_diff = cmd_time_stamp[1:] - cmd_time_stamp[:-1]
    calc_u_opt_time = np.loadtxt(dir_name+"/debug_mpc_calc_u_opt_time.csv",delimiter=',')
    total_ctrl_time = np.loadtxt(dir_name+"/debug_mpc_total_ctrl_time.csv",delimiter=',')
    start_time= control_cmd[0,0]+1e-9*control_cmd[0,1]
    total_time_default=control_cmd[-1,0]+1e-9*control_cmd[-1,1]-start_time

    plt.plot(cmd_time_stamp[1:],cmd_time_stamp_diff,label="/control/trajectory_follower/control_cmd publish_interval")
    plt.plot(total_ctrl_time[:,0] + 1e-9*total_ctrl_time[:,1],total_ctrl_time[:,2],label="total_ctrl_time")
    plt.plot(calc_u_opt_time[:,0] + 1e-9*calc_u_opt_time[:,1],calc_u_opt_time[:,2],label="calc_u_opt_time")
    plt.legend()
    plt.ylim(ylim)
    if total_time is None:
        plt.xlim([start_time+skip_time,start_time+total_time_default])
    else:
        plt.xlim([start_time+skip_time,start_time+skip_time+total_time])
    plt.xlabel("Time [s]")
    plt.ylabel("Processing Time [s]")
    plt.show()

In [ ]:
dirname = #specify rosbag dir
steer_visualize(20,20,dirname,ylim=[-0.05,0.05])
lateral_error_visualize(0,None,dirname,ylim=[-0.6,0.6])
vel_visualize(20,20,dirname,ylim=[-0.6,12.0])